# Classification using BFGS -- Pytorch version

This notebook details the implementation of a generic ridge-regularized classification solved by direct gradient-based optimization (here quasi-newton). 
It is implemented in the kernel space, i.e. representing the weights over the space of points.

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")
device_cpu = device
print( device )

mytype = torch.float16 # to save memory (only on GPU)
mytype = torch.float32

cpu


# Data

## Kprot

In [12]:
import pickle
with open('data/CC_all2_base_K_prot.data', 'rb') as f:
        K_prot = pickle.load(f)

In [13]:
K_prot.shape

(1647, 1647)

## liste des 152 844 smiles

In [3]:
import pandas as pd
import zipfile
base_name = 'Consensus_CompoundBioactivity_Dataset_v1.1_Sh2_all2'
zf = zipfile.ZipFile('data/CC/' + base_name + '.csv.zip') 
df = pd.read_csv(zf.open(base_name + '.csv'),low_memory=False)
df.rename(columns={'standardized smiles':'smiles'}, inplace=True)
df.head()

,smiles,fasta,ChEMBL ID,PubChem ID,IUPHAR ID,Ligand names,Target,uniprot,mean,pIC50,pKi,pKd,measure,score,interaction+,indsmiles,indfasta
0,BrC1CCN(Cc2ccc(OCCCN3CCCCC3)cc2)CC1,MERAPPDGPLNASGALAGEAAAAGGARGFSAAWTAVLAALMALLIV...,CHEMBL472466,44592131.0,NaN,4-bromo-1-(4-(3-(piperidin-1-yl)propoxy)benzyl...,hrh3,Q9Y5N1,9.7,NaN,9.7,NaN,9.7,1.0,True,0.0,0.0
1,BrCC(Br)C1CCC(Br)C(Br)C1,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,CHEMBL375107,18728.0,NaN,"1,2-dibromo-4-(1,2-dibromoethyl)cyclohexane\nn...",ar,P10275,7.4,7.4,NaN,NaN,7.4,1.0,True,1.0,1.0
2,Brc1c(C2=CN3CCC2CC3)oc2ccccc12,MANFTPVNGSSGNQSVRLVTSSSHNRYETVEMVFIATVTGSLSLVT...,CHEMBL415510,10494696.0,NaN,3-(3-bromo-benzofuran-2-yl)-1-aza-bicyclo[2.2....,chrm4,P08173,7.0,NaN,7.0,NaN,7.0,1.0,True,2.0,2.0
3,Brc1c(C2=CN3CCC2CC3)oc2ccccc12,MEGDSYHNATTVNGTPVNHQPLERHRLWEVITIAAVTAVVSLITIV...,CHEMBL415510,10494696.0,NaN,3-(3-bromo-benzofuran-2-yl)-1-aza-bicyclo[2.2....,chrm5,P08912,7.0,NaN,7.0,NaN,7.0,1.0,True,2.0,3.0
4,Brc1c(C2=CN3CCC2CC3)oc2ccccc12,MNNSTNSSNNSLALTSPYKTFEVVFIVLVAGSLSLVTIIGNILVMV...,CHEMBL415510,10494696.0,NaN,3-(3-bromo-benzofuran-2-yl)-1-aza-bicyclo[2.2....,chrm2,P08172,7.0,NaN,7.0,NaN,7.0,1.0,True,2.0,4.0


In [4]:
df_p = df[df['interaction+'] == True]
#list of smiles strings
dict_ind2smiles = df_p[['indsmiles','smiles']].set_index('indsmiles').to_dict()['smiles']
# ordonner les indices suivant les ind2mol
dict_ind2smiles = {k:dict_ind2smiles[k] for k in sorted(dict_ind2smiles.keys())}
print(len(dict_ind2smiles))
smiles = list(dict_ind2smiles.values())
len(smiles)

152844


152844

In [2]:
# same in zip format
import pandas as pd
import zipfile
zf = zipfile.ZipFile('data/CC/Consensus_CompoundBioactivity_Dataset_v1.1_Sh2_all2.csv.zip') 
df = pd.read_csv(zf.open('Consensus_CompoundBioactivity_Dataset_v1.1_Sh2_all2.csv'),low_memory=False)
df_p = df[df['interaction+'] == True]
#list of smiles strings
smiles = df_p['standardized smiles'].drop_duplicates().values
len(smiles)

152844

In [15]:
from rdkit import Chem
from rdkit.Chem import AllChem

import numpy as np

nM = 10000 # len(smiles)
MorganFP = np.zeros((nM,1024))
for i in range(nM):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles[i])    
    # Generate Morgan fingerprint of the molecule
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    # Convert the fingerprint to a numpy array
    arr = np.zeros((1,))
    AllChem.DataStructs.ConvertToNumpyArray(fp, arr)
    MorganFP[i,:] = arr
MorganFP = MorganFP.astype(int)

In [16]:
import Nystrom_method
from  Nystrom_method import nystroem,KronKernel
# random list of molecules 
kM = 3000 # number of molecule to compute nystrom
rM = 1000 # final dimension of features
I = np.random.permutation(nM)
I = I[:kM]

In [17]:
# compute Tanimoto kernel 
Km = ( MorganFP[I,:] @ MorganFP.T ) / ( 1024 - (1-MorganFP[I,:]) @ (1-MorganFP.T) )

In [18]:
Xm,Lambda,LambdaC = nystroem(np.concatenate((Km[:,I], Km), axis=1),rM)

## liste des indices protéines/molécules avec que des 1

In [5]:
# protein indices
J = df_p['indfasta'].values
print(len(J))
# molecules indices
I = df_p['indsmiles'].values
print(len(I))

231964
231964


## train/test avec indices protéines/molécules et interactions balanced

In [8]:
# load with pickle
import pickle
with open('data/train.data', 'rb') as f:
        train = pickle.load(f)
with open('data/test.data', 'rb') as f:
        test = pickle.load(f)

print(len(train))

371142


In [9]:
# en premier l'indice de la protéine, puis l'indice du ligand puis l'interaction
train

array([[   145, 124184,      1],
       [   145, 125039,      1],
       [   145,  48320,      1],
       ...,
       [  1643, 150939,      0],
       [  1647, 152928,      1],
       [  1647, 126691,      0]])

In [20]:
from utils import make_train_test
all_train_interactions_arr, all_test_interactions_arr = make_train_test(df,5,1)

train (366418, 3)
test (91606, 3)
train (366418, 3)
test (91606, 3)
train (366420, 3)
test (91604, 3)
train (366420, 3)
test (91604, 3)
train (366420, 3)
test (91604, 3)
Train/test datasets prepared.


In [21]:
with open('data/CC_train_arr.data', 'wb') as f:
    pickle.dump(all_train_interactions_arr, f)
with open('data/CC_test_arr.data', 'wb') as f:
    pickle.dump(all_test_interactions_arr, f)

In [5]:
from utils import make_train_test_prot_orphan

all_train_interactions_arr, all_test_interactions_arr = make_train_test_prot_orphan(df,5)

train (366418, 3)
test (91606, 3)
train (366418, 3)
test (91606, 3)
train (366420, 3)
test (91604, 3)
train (366420, 3)
test (91604, 3)
train (366420, 3)
test (91604, 3)
Train/test datasets prepared.


In [6]:
from utils import make_train_test_mol_orphan

all_train_interactions_arr, all_test_interactions_arr = make_train_test_mol_orphan(df,5)

train (366418, 3)
test (91606, 3)
train (366418, 3)
test (91606, 3)
train (366420, 3)
test (91604, 3)
train (366420, 3)
test (91604, 3)
train (366420, 3)
test (91604, 3)
Train/test datasets prepared.


In [3]:
%load_ext autoreload
%autoreload 2

import utils
from utils import make_train_test_prot_mol_orphan

all_train_interactions_arr, all_test_interactions_arr = make_train_test_prot_mol_orphan(df,5)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
train (184470, 3)
test (36376, 3)
train (202894, 3)
test (30838, 3)
train (195680, 3)
test (32855, 3)
train (192080, 3)
test (33824, 3)
train (186702, 3)
test (35373, 3)
Train/test datasets prepared.


In [4]:
with open('data/CC/train_arr_prot_mol_orphan.data', 'wb') as f:
    pickle.dump(all_train_interactions_arr, f)
with open('data/CC/test_arr_prot_mol_orphan.data', 'wb') as f:
    pickle.dump(all_test_interactions_arr, f)

NameError: name 'pickle' is not defined

In [12]:
S_train = set(map(tuple, train))
S_test = set(map(tuple, test))
S_train.intersection(S_test)

NameError: name 'train' is not defined

In [ ]:
for elt in test:
    for x in train:
        if elt[0]==x[0]:
            print(elt,x)

In [ ]:
for elt in test:
    for x in train:
        if elt[1]==x[1]:
            print(elt,x)